### Maximal Marginal Relevance
MMR (Maximal Marginal Relevance) is a powerful diversity-aware retrieval technique used in information retrieval and RAG pipelines to balance relevance and novelty when selecting documents.

In [17]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

In [18]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


In [19]:
#Step 1: Load and chunk the document
loader = TextLoader("langchain_rag_dataset.txt")
raw_docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap=50)
chunks = splitter.split_documents(raw_docs)
chunks

[Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain is an open-source framework designed to simplify the development of applications using large language models (LLMs).\nLangChain provides abstractions for working with prompts, chains, memory, and agents, making it easier to build complex LLM-based systems.'),
 Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='The framework supports integration with various vector databases like FAISS and Chroma for semantic retrieval.\nLangChain enables Retrieval-Augmented Generation (RAG) by allowing developers to fetch relevant context before generating responses.'),
 Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.'),
 Document(metadata={'

In [20]:
## Step 2 : FIASS vector store with Hugging Face embeddings 

embedding_model = HuggingFaceEmbeddings(model = "all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding_model)

In [21]:
### Step 3: Create MMR Retriever 
retriever = vectorstore.as_retriever(
    search_type = "mmr", 
    seatch_kwargs = {"k":3}

)

In [22]:
### Step 4: Prompt and LLM

prompt = PromptTemplate.from_template("""
Answer the question based on the context provided.
Context:
{context}

Question:{input}
""")

llm = init_chat_model("groq:gemma2-9b-it")

In [23]:
### step 5: RAG pipeline
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain = create_retrieval_chain(retriever=retriever, combine_docs_chain=document_chain)

In [24]:
#Step 6: Query 
query = {"input":"How does Langchain support agents and memory?"}
response = rag_chain.invoke(query)

In [25]:

print("Answer:\n", response['answer'])

Answer:
 LangChain supports agents in two key ways:

1. **Tool Integration:** Agents can utilize various tools like calculators, search APIs, or custom functions based on the instructions they receive. This allows them to perform tasks beyond simple text generation.

2. **Decision Making:** LangChain empowers LLMs within agents to act as decision-makers. They can choose which tool to employ and the sequence in which to use them while completing a task.

Regarding memory, LangChain offers two primary memory options:

1. **Conversational Memory:**  `ConversationBufferMemory` enables agents to retain previous interactions, ensuring coherence in multi-turn conversations.

2. **Summarization Memory:** `ConversationSummaryMemory` condenses past interactions into summaries, providing a concise overview for the agent to reference. 


This combination of tool integration and memory support allows LangChain agents to be more capable and context-aware. 



In [26]:
response

{'input': 'How does Langchain support agents and memory?',
 'context': [Document(id='abc23e29-fca0-4b8c-8f8b-6cb5367eca0c', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.'),
  Document(id='9769c4c7-848e-4908-bd36-7ea05ac09ba1', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain agents can interact with external APIs and databases, enhancing the capabilities of LLM-powered applications.\nRAG pipelines in LangChain involve document loading, splitting, embedding, retrieval, and LLM-based response generation.'),
  Document(id='d0cceef8-94ae-43d2-9cd0-3579c94bebfe', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain allows LLMs to act as agents that decide which tool to call and in what order duri